In [1]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
import time
from atcs import ATCS
import fig_plot as fp


In [2]:
# N_sample = 4
# seed = 1
# solver_type = 'minlp'
# model_type = 'stochastic'
# sample_type = f'head_{N_sample}'
# data = ATCS(seed = 1)
# data.choose_subset_point(N_sample, randomized = False) # Choose subset data

In [3]:
# data.r_s_sub_df

In [4]:
# location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
# location_fig.show()

In [5]:
import pandas as pd
import numpy as np
import time
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS

def run_minlp_stochastic(data:ATCS, N_sample, sample_type, seed = 1,save = True):
    # Stochastic MINLP Model
    # Data
    robot_id = data.r_s_sub_df.index.to_numpy()
    robot_loc = data.l_sub_df.to_numpy()
    # robot_range = data.R_Kub_df.to_numpy().flatten()
    x_min,x_max = np.min(robot_loc, axis=0)[0], np.max(robot_loc, axis=0)[0]
    y_min,y_max = np.min(robot_loc, axis=0)[1], np.max(robot_loc, axis=0)[1]
    # dist_matrix = data.get_distance_matrix(sample_subset = True)

    # Creates an empty Model
    model = xp.problem(name='MINLP_Stochastic')
    # Given Inputs and Parameters
    R_K = {(i, col): data.r_s_sub_df.at[i, col] for i in data.r_s_sub_df.index for col in data.r_s_sub_df.columns}
    L_Range = data.r_min # km
    U_Range = data.r_max # km
    U_Robot = data.q # Max robots per charger
    U_Charger = data.m # Max chargers per station
    L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station
    Dist_Max = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
    C_b = data.c_b # Investment cost per station
    C_h = data.c_h # Cost of moving a robot
    C_m = data.c_m # Maintenance cost per charger
    C_c = data.c_c # Charging cost per km

    Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()
    A_dict = {(i, col): data.cc_df.at[i, col] for i in data.cc_df.index for col in data.cc_df.columns}

    # Parameters
    # U_Station = L_Station + 1 # For now
    U_Station = len(robot_id)
    data.set_output_folder(solver_type, model_type, name = f'{N_sample}_{sample_type}samp_seed{seed}')

    # Sets and Notation
    V = data.r_s_sub_df.index.to_numpy() # Robot Vehicle sets
    S = np.arange(U_Station)
    K = data.r_s_sub_df.columns.to_list()


    # Output Variables
    x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = x_min, ub = x_max) # Longitude of station s
    y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = y_min, ub = y_max) # Latitude of station s

    eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
    mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
    beta = model.addVariables(V, S, K, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
    alpha = model.addVariables(V, S, K, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
    d = model.addVariables(V, S, K, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


    # Robot v can be allocated to charging station s iff it is built
    model.addConstraint(beta[v,s,k] <= A_dict[v,k] * mu[s] for v in V for s in S for k in K) 

    model.addConstraint(xp.Sum(mu[s] for s in S) >= L_Station)
    # model.addConstraint(xp.Sum(mu[s] for s in S) <= L_Station + 1)
    # Each Robot v must be allocated to only one station
    model.addConstraint(xp.Sum(beta[v,s,k] for s in S) == 1 * A_dict[v,k] for v in V for k in K)

    # Number of chargers per station
    model.addConstraint(eta[s] >= xp.Sum(A_dict[v,k]*beta[v,s,k] for v in V)/U_Robot for s in S for k in K)
    model.addConstraint(eta[s] <= U_Charger for s in S)

    # Robot v can be brought to station s by human iff the robot is allocated to that station
    model.addConstraint(alpha[v,s,k] <= beta[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)

    model.addConstraint(d[v,s,k] <= R_K[v,k] * (1 - alpha[v,s,k]) * A_dict[v,k] for v in V for s in S for k in K)
    model.addConstraint(d[v,s,k] >= A_dict[v,k]*(xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
                                    - Dist_Max * (alpha[v,s,k] + (1 - beta[v,s,k]))) 
                                            for v in V for s in S for k in K)
    model.addConstraint(d[v,s,k] <= Dist_Max * beta[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)

    # Objective Function
    Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
    Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)

    Penalty_cost = (1/len(K))*xp.Sum(C_h * alpha[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)
    Charging_cost = (1/len(K))*xp.Sum(C_c * A_dict[v,k]*(beta[v,s,k]*U_Range - (beta[v,s,k]*R_K[v,k] - d[v,s,k])) for v in V for s in S for k in K)

    obj = Build_station_cost + Build_charger_cost + Charging_cost + Penalty_cost

    model.setObjective(obj, sense=xp.minimize)
    model.setControl('maxtime', 3600)
    
    # Solve the problem
    tic = time.time()
    model.solve()
    toc = time.time()
    run_time = tic - toc
    
    # Preprocess the Output
    obj_value = model.getObjVal()
    mu_dict = model.getSolution(mu)
    eta_dict = model.getSolution(eta)
    x_dict = model.getSolution(x)
    y_dict = model.getSolution(y)
    beta_dict = model.getSolution(beta)
    alpha_dict = model.getSolution(alpha)
    d_dict = model.getSolution(d)

    summary_df = pd.DataFrame({'Objective':[obj_value], 'RunTime':[run_time]})

    station_df = pd.DataFrame([mu_dict, eta_dict, x_dict, y_dict], ['mu','eta','longitude','latitude']).T
    station_df['Building_Station_Cost'] = station_df['mu'] * C_b
    station_df['Building_Charger_Cost'] = station_df['eta'] * C_m

    r_k_df = pd.DataFrame.from_dict(R_K, orient='index')
    r_k_df.rename(columns={0:'range_scenario'}, inplace=True)

    A_df = pd.DataFrame.from_dict(A_dict, orient='index')
    A_df.rename(columns={0:'A_v_k'}, inplace = True)

    robot_df = pd.DataFrame([beta_dict, alpha_dict, d_dict],['beta','alpha','d']).T.reset_index()
    robot_df.rename(columns={'index':'(v,s,k)'}, inplace=True)
    robot_df['v'], robot_df['s'], robot_df['k'] = robot_df['(v,s,k)'].apply(lambda x: x[0]), robot_df['(v,s,k)'].apply(lambda x: x[1]), robot_df['(v,s,k)'].apply(lambda x: x[2])
    robot_df['(v,k)'] = robot_df['(v,s,k)'].apply(lambda x: (x[0],x[2]))
    robot_df.drop(['(v,s,k)'], axis = 1, inplace= True)
    robot_df = robot_df.set_index('(v,k)').merge(r_k_df, left_index = True, right_index = True, how = 'left')
    robot_df = robot_df.merge(A_df, left_index = True, right_index = True, how = 'left')
    robot_df.reset_index(drop = True, inplace=True)
    
    if save:
        station_df.to_csv(f'{data.folder_name}/station_df.csv')
        robot_df.to_csv(f'{data.folder_name}/robot_df.csv')

    return summary_df, station_df, robot_df, mu_dict, eta_dict, x_dict, y_dict, beta_dict, alpha_dict, d_dict


if __name__ == "__main__":
    N_sample = 10
    seed = 1
    solver_type = 'minlp'
    model_type = 'stochastic'
    sample_type = f'head_{N_sample}'
    data = ATCS(seed = 1)
    data.choose_subset_point(N_sample, randomized = False) # Choose subset data

    # (summary_df, station_df, robot_df, 
    # mu_dict, eta_dict, x_dict, y_dict, 
    # beta_dict, alpha_dict, d_dict) = run_minlp_stochastic(data, N_sample, sample_type, seed = 1,save = True)


In [6]:
asdfasdf

NameError: name 'asdfasdf' is not defined

In [ ]:
# summary_df, station_df, robot_df, mu_dict, eta_dict, x_dict, y_dict, beta_dict, alpha_dict, d_dict = run_minlp_stochastic(data)

Original problem size
   linear:    8806 rows, 4816 columns, 14572 linear coefficients
   nonlinear: 1600 coefficients, 49600 tokens
             11200 mul             0 div      1600 sqrt
Nonlinear presolve
   linear presolve removed 3849 rows, 688 columns, 5056 linear coefficients
   removed 1376 fixed variables from formulas
   simplify removed 5504 tokens
   creating 2512 '+' clusters removed 3200 tokens
Presolved problem size
   linear:    4957 rows, 4129 columns, 9516 linear coefficients
   nonlinear: 1600 coefficients, 40896 tokens
              6624 mul             0 div      1600 sqrt
                 0 exp             0 log      3200 pow
Problem is nonlinear presolved
FICO Xpress v9.4.2, Hyper, solve started 19:41:01, Apr 7, 2025
Control settings used:
MAXTIME = 3600
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Maximum expanded nl-formula size: 29  (row 'R5609')
Total tokens: 40896
  11  parallel calculation threads
  Jacobian: symbolic differenti

In [ ]:
# # Stochastic MINLP Model
# # Data
# robot_id = data.r_s_sub_df.index.to_numpy()
# robot_loc = data.l_sub_df.to_numpy()
# # robot_range = data.R_Kub_df.to_numpy().flatten()
# x_min,x_max = np.min(robot_loc, axis=0)[0], np.max(robot_loc, axis=0)[0]
# y_min,y_max = np.min(robot_loc, axis=0)[1], np.max(robot_loc, axis=0)[1]
# # dist_matrix = data.get_distance_matrix(sample_subset = True)

# # Creates an empty Model
# model = xp.problem(name='MINLP_Stochastic')
# # Given Inputs and Parameters
# R_K = {(i, col): data.r_s_sub_df.at[i, col] for i in data.r_s_sub_df.index for col in data.r_s_sub_df.columns}
# L_Range = data.r_min # km
# U_Range = data.r_max # km
# U_Robot = data.q # Max robots per charger
# U_Charger = data.m # Max chargers per station
# L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station
# Dist_Max = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
# C_b = data.c_b # Investment cost per station
# C_h = data.c_h # Cost of moving a robot
# C_m = data.c_m # Maintenance cost per charger
# C_c = data.c_c # Charging cost per km

# Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()
# A_dict = {(i, col): data.cc_df.at[i, col] for i in data.cc_df.index for col in data.cc_df.columns}

# # Parameters
# # U_Station = L_Station + 1 # For now
# U_Station = len(robot_id)
# data.set_output_folder(solver_type, model_type, name = f'{N_sample}_{sample_type}samp_{U_Station}s_seed{seed}')

# # Sets and Notation
# V = data.r_s_sub_df.index.to_numpy() # Robot Vehicle sets
# S = np.arange(U_Station)
# K = data.r_s_sub_df.columns.to_list()


# # Output Variables
# x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = x_min, ub = x_max) # Longitude of station s
# y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = y_min, ub = y_max) # Latitude of station s

# eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
# mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
# beta = model.addVariables(V, S, K, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
# alpha = model.addVariables(V, S, K, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
# d = model.addVariables(V, S, K, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


# # Robot v can be allocated to charging station s iff it is built
# model.addConstraint(beta[v,s,k] <= A_dict[v,k] * mu[s] for v in V for s in S for k in K) 

# model.addConstraint(xp.Sum(mu[s] for s in S) >= L_Station)
# model.addConstraint(xp.Sum(mu[s] for s in S) <= L_Station + 1)
# # Each Robot v must be allocated to only one station
# model.addConstraint(xp.Sum(beta[v,s,k] for s in S) == 1 * A_dict[v,k] for v in V for k in K)

# # Number of chargers per station
# model.addConstraint(eta[s] >= xp.Sum(A_dict[v,k]*beta[v,s,k] for v in V)/U_Robot for s in S for k in K)
# model.addConstraint(eta[s] <= U_Charger for s in S)

# # Robot v can be brought to station s by human iff the robot is allocated to that station
# model.addConstraint(alpha[v,s,k] <= beta[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)

# model.addConstraint(d[v,s,k] <= R_K[v,k] * (1 - alpha[v,s,k]) * A_dict[v,k] for v in V for s in S for k in K)
# model.addConstraint(d[v,s,k] >= xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
#                                 - Dist_Max * (alpha[v,s,k] + (1 - beta[v,s,k])) 
#                                         for v in V for s in S for k in K)
# model.addConstraint(d[v,s,k] <= Dist_Max * beta[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)

# # Objective Function
# Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
# Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)

# Penalty_cost = (1/len(K))*xp.Sum(C_h * alpha[v,s,k] * A_dict[v,k] for v in V for s in S for k in K)
# Charging_cost = (1/len(K))*xp.Sum(C_c * A_dict[v,k]*(beta[v,s,k]*U_Range - (beta[v,s,k]*R_K[v,k] - d[v,s,k])) for v in V for s in S for k in K)

# obj = Build_station_cost + Build_charger_cost + Charging_cost + Penalty_cost

# model.setObjective(obj, sense=xp.minimize)
# # model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', 3600)
# # Solve the problem
# model.solve()

In [ ]:
mu_dict = model.getSolution(mu)
eta_dict = model.getSolution(eta)
x_dict = model.getSolution(x)
y_dict = model.getSolution(y)

station_df = pd.DataFrame([mu_dict, eta_dict, x_dict, y_dict], ['mu','eta','longitude','latitude']).T
station_df['Building_Station_Cost'] = station_df['mu'] * C_b
station_df['Building_Charger_Cost'] = station_df['eta'] * C_m

NameError: name 'model' is not defined

In [ ]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,1.0,2.0,-122.679901,-81.120257,5000.0,1000.0
1,0.0,0.0,-134.537839,-84.681705,0.0,0.0
2,0.0,0.0,-134.537839,-84.681705,0.0,0.0
3,0.0,0.0,-134.537839,-84.681705,0.0,0.0


In [ ]:
r_k_df = pd.DataFrame.from_dict(R_K, orient='index')
r_k_df.rename(columns={0:'range_scenario'}, inplace=True)

A_df = pd.DataFrame.from_dict(A_dict, orient='index')
A_df.rename(columns={0:'A_v_k'}, inplace = True)

In [ ]:
beta_dict = model.getSolution(beta)
alpha_dict = model.getSolution(alpha)
d_dict = model.getSolution(d)

robot_df = pd.DataFrame([beta_dict, alpha_dict, d_dict],['beta','alpha','d']).T.reset_index()
robot_df.rename(columns={'index':'(v,s,k)'}, inplace=True)
robot_df['v'], robot_df['s'], robot_df['k'] = robot_df['(v,s,k)'].apply(lambda x: x[0]), robot_df['(v,s,k)'].apply(lambda x: x[1]), robot_df['(v,s,k)'].apply(lambda x: x[2])
robot_df['(v,k)'] = robot_df['(v,s,k)'].apply(lambda x: (x[0],x[2]))
robot_df.drop(['(v,s,k)'], axis = 1, inplace= True)
robot_df = robot_df.set_index('(v,k)').merge(r_k_df, left_index = True, right_index = True, how = 'left')
robot_df = robot_df.merge(A_df, left_index = True, right_index = True, how = 'left')
robot_df.reset_index(drop = True, inplace=True)
# robot_df = robot_df[['v','s','k','beta','alpha','d']]
# robot_df = robot_df.groupby(['v','k','s']).sum()

In [ ]:
C_h = data.c_h
C_c = data.c_c
U_Range = data.r_max
K = data.r_s_sub_df.columns.to_list()

In [ ]:
robot_df['Penalty_Cost'] = robot_df['alpha'] * robot_df['A_v_k'] * C_h
robot_df['Charging_Cost'] = C_c * robot_df['A_v_k'] * (robot_df['beta']*U_Range - (robot_df['beta']*robot_df['range_scenario'] - robot_df['d']))
# Charging_cost = (1/len(K))*xp.Sum(C_c * A_dict[v,k]*(beta[v,s,k]*U_Range - (beta[v,s,k]*R_K[v,k] - d[v,s,k])) for v in V for s in S for k in K)


In [ ]:
(1/len(K))*(robot_df['Penalty_Cost'].sum()) + (1/len(K))*(robot_df['Charging_Cost'].sum()) + 5000 + 1000

6141.499052206885

In [ ]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,1.0,2.0,-122.679901,-81.120257,5000.0,1000.0
1,0.0,0.0,-134.537839,-84.681705,0.0,0.0
2,0.0,0.0,-134.537839,-84.681705,0.0,0.0
3,0.0,0.0,-134.537839,-84.681705,0.0,0.0


In [ ]:
r_k_df = pd.DataFrame.from_dict(R_K, orient='index').reset_index()

NameError: name 'R_K' is not defined

In [ ]:
r_k_df

,index,0
0,"(0, s0)",127.868274
1,"(0, s1)",99.607518
2,"(0, s2)",115.991472
3,"(0, s3)",98.862235
4,"(0, s4)",56.717849
...,...,...
395,"(3, s95)",58.621838
396,"(3, s96)",57.366586
397,"(3, s97)",67.822953
398,"(3, s98)",19.142513


In [ ]:
robot_df[(robot_df['v'] == 0)]

,beta,alpha,d,v,s,k,range_scenario,A_v_k,Penalty_Cost,Charging_Cost
0,0.0,0.0,0.000000,0,0,s0,127.868274,0,0.0,0.000000
1,0.0,0.0,0.000000,0,0,s1,99.607518,0,0.0,0.000000
2,1.0,0.0,3.080686,0,0,s2,115.991472,1,0.0,26.077470
3,0.0,0.0,0.000000,0,0,s3,98.862235,0,0.0,0.000000
4,1.0,0.0,3.080686,0,0,s4,56.717849,1,0.0,50.972391
...,...,...,...,...,...,...,...,...,...,...
395,0.0,0.0,0.000000,0,3,s95,85.335774,1,0.0,0.000000
396,0.0,0.0,0.000000,0,3,s96,45.937307,1,0.0,0.000000
397,0.0,0.0,0.000000,0,3,s97,40.648336,1,0.0,0.000000
398,0.0,0.0,0.000000,0,3,s98,75.513443,1,0.0,0.000000


In [ ]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,1.0,2.0,-122.679901,-81.120257,5000.0,1000.0
1,0.0,0.0,-134.537839,-84.681705,0.0,0.0
2,0.0,0.0,-134.537839,-84.681705,0.0,0.0
3,0.0,0.0,-134.537839,-84.681705,0.0,0.0


In [7]:
station_df = pd.read_csv('minlp_output/stochastic/10_head_10samp_seed1/station_df.csv', index_col=0)

In [8]:
heu_station_df = pd.DataFrame([(-125.4642114026697, -83.34843878988191)])
heu_station_df.columns = ['longitude','latitude']

In [10]:
location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig = fp.add_point_to_l_fig(location_fig, station_df[station_df['mu'] != 0], size=7, name='MINLP Stations', color='green')
location_fig = fp.add_point_to_l_fig(location_fig, heu_station_df, size=7, name='Construction Heuristic Stations', color='Blue')
# # location_fig = fp.add_point_to_l_fig(location_fig, cen_heu_station_df, size=7, name='Centroid Heuristic Stations', color='Yellow')
# location_fig = fp.add_point_to_l_fig(location_fig, imp_heu_station_df, size=7, name='Improved Heuristic Stations', color='Purple')
location_fig.update_layout(geo=dict(
            scope="world",  # Or use 'europe', etc.
            showland=True,
            lataxis=dict(range=[-180,-150]),  # Set latitude range (y-axis)
            lonaxis=dict(range=[-180,180])   # Optional: set longitude range (x-axis)
        ),        legend=dict(
            x=0.63,
            y=1.00,
            bgcolor='rgba(255,255,255,0.7)',
            # bordercolor='black',
            # borderwidth=1
        ))
location_fig.show()

In [ ]:
# E[(Rmax - Rk)*Ik] = (1/2)(175 - 10)*1 + (1/2)(175 - 50) * 1

In [ ]:
# Each Robot 1 Weight Centroid
# (1/2)*(175 - 10)*1 + (1/2)*(175 - 50) * 0

In [ ]:
Rmax - E[Rk * I]

NameError: name 'Rmax' is not defined